## 1. Describe SimLex data. (15)

In [73]:
import stanza
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score

In [74]:
simlex_data = pd.read_csv("SimLex-999/SimLex-999.txt", sep="\t", header=0)
simlex_data

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [75]:
max([len(x) for x in simlex_data['word1']])

14

In [76]:
max([len(x) for x in simlex_data['word2']])

12

In [77]:
min([len(x) for x in simlex_data['word1']])

2

In [78]:
min([len(x) for x in simlex_data['word2']])

2

In [79]:
# Display basic information about the dataset
print("SimLex999 Dataset Description:")
print("Number of word pairs:", len(simlex_data))
print()
print("Columns:", simlex_data.columns)
print()
print("Sample data:")
simlex_data.head()

SimLex999 Dataset Description:
Number of word pairs: 999

Columns: Index(['word1', 'word2', 'POS', 'SimLex999', 'conc(w1)', 'conc(w2)', 'concQ',
       'Assoc(USF)', 'SimAssoc333', 'SD(SimLex)'],
      dtype='object')

Sample data:


,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93


In [80]:
set(simlex_data['POS'])

{'A', 'N', 'V'}

In [81]:
simlex_data.shape

(999, 10)

In [82]:
simlex_data.describe()

,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,4.561572,3.657087,3.568629,2.501502,0.751512,0.333333,1.274505
std,2.614663,1.131050,1.159572,1.118145,1.344569,0.471641,0.366278
min,0.230000,1.190000,1.190000,1.000000,0.000000,0.000000,0.340000
25%,2.380000,2.620000,2.500000,2.000000,0.140000,0.000000,1.075000
50%,4.670000,3.830000,3.660000,3.000000,0.250000,0.000000,1.310000
75%,6.750000,4.790000,4.750000,3.500000,0.680000,1.000000,1.540000
max,9.800000,5.000000,5.000000,4.000000,8.850000,1.000000,2.180000


In [83]:
simlex_data = simlex_data[['word1', 'word2', 'POS']]
simlex_data

,word1,word2,POS
0,old,new,A
1,smart,intelligent,A
2,hard,difficult,A
3,happy,cheerful,A
4,hard,easy,A
...,...,...,...
994,join,acquire,V
995,send,attend,V
996,gather,attend,V
997,absorb,withdraw,V


The dataset contains some subset of simlex reviews. There are 999 items of pairs in the dataset.

1. Hill, Felix, Roi Reichart, and Anna Korhonen. "Simlex-999: Evaluating semantic models with (genuine) similarity estimation." Computational Linguistics 41.4 (2015): 665-695.
2. Bojanowski, Piotr, Edouard Grave, Armand Joulin, and Tomas Mikolov. "Enriching word vectors with subword information." Transactions of the association for computational linguistics 5 (2017): 135-146.
3. Mikolov, Tomas, Edouard Grave, Piotr Bojanowski, Christian Puhrsch, and Armand Joulin. "Advances in pre-training distributed word representations." arXiv preprint arXiv:1712.09405 (2017).

## 2. Calculate word similarities based on WordNet’s path_similarity (iterate over all synsets pairs the words belong to, account for POS tags). Report number of word pairs, where one of the words is missing in WordNet.

In [84]:
from nltk.corpus import wordnet as wn

count_of_words_with_missings = 0

# Function to calculate word similarity using WordNet's path_similarity
def word_similarity_wordnet(word1, word2):
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)
    
    if not synsets1 or not synsets2:
        print(word1, word2)
        count_of_words_with_missings += 1
        return 
    
    max_similarity = 0.0
    
    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.path_similarity(synset2)
            if similarity is not None and similarity > max_similarity:
                max_similarity = similarity
                
    return max_similarity


word1 = "dog"
word2 = "cat"
similarity = word_similarity_wordnet(word1, word2)
print(f"WordNet Similarity between '{word1}' and '{word2}': {similarity}")

WordNet Similarity between 'dog' and 'cat': 0.2


In [85]:
simlex_data['path_similarity'] = [word_similarity_wordnet(row['word1'], row['word2']) for _, row in simlex_data.iterrows()]

/var/folders/9p/vfj51s1x3f7gl6dvkpvb09mm0000gn/T/ipykernel_42659/121983393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simlex_data['path_similarity'] = [word_similarity_wordnet(row['word1'], row['word2']) for _, row in simlex_data.iterrows()]


In [86]:
simlex_data[simlex_data['path_similarity'] == 0.05263157894736842]

,word1,word2,POS,path_similarity
225,dictionary,definition,N,0.052632


In [87]:
min(simlex_data['path_similarity'])

0.05263157894736842

In [88]:
simlex_data[simlex_data['path_similarity'] == 1]

,word1,word2,POS,path_similarity
2,hard,difficult,A,1.0
6,happy,glad,A,1.0
19,hard,tough,A,1.0
20,new,fresh,A,1.0
24,wonderful,terrific,A,1.0
...,...,...,...,...
839,acquire,get,V,1.0
843,take,carry,V,1.0
861,steal,buy,V,1.0
885,declare,announce,V,1.0


In [89]:
max(simlex_data['path_similarity'])

1.0

In [90]:
count_of_words_with_missings

0

# 3. Download English fastText model in binary format (https://fasttext.cc/docs/en/crawlvectors.html). 

# Calculate word similarities based on cosine similarity of word vectors (note that e.g. scipy.spatial.distance.cosine returns 1 – cos(u,v) ). 

# Report if any words are missing in the model.

In [91]:
pip install fasttext

python(44043) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [92]:
import fasttext.util
import fasttext

fasttext.util.download_model('en', if_exists='ignore')
model = fasttext.load_model('cc.en.300.bin')

def word_similarity_fasttext(word1, word2):
    try:
        vector1 = model.get_word_vector(word1)
        vector2 = model.get_word_vector(word2)
        cosine_similarity = 1 - scipy.spatial.distance.cosine(vector1, vector2)
        return cosine_similarity
    except KeyError:
        return None

In [93]:
simlex_data['cos_similarity'] = [word_similarity_fasttext(row['word1'], row['word2']) for _, row in simlex_data.iterrows()]

/var/folders/9p/vfj51s1x3f7gl6dvkpvb09mm0000gn/T/ipykernel_42659/222330704.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simlex_data['cos_similarity'] = [word_similarity_fasttext(row['word1'], row['word2']) for _, row in simlex_data.iterrows()]


In [94]:
min(simlex_data['cos_similarity'])

0.022019339725375175

In [95]:
simlex_data[simlex_data['cos_similarity'] == 0.022019339725375175]

,word1,word2,POS,path_similarity,cos_similarity
762,deck,mouse,N,0.166667,0.022019


In [96]:
max(simlex_data['cos_similarity'])

0.9787169098854065

In [97]:
simlex_data[simlex_data['cos_similarity'] == 0.9787169098854065]

,word1,word2,POS,path_similarity,cos_similarity
115,south,north,N,0.333333,0.978717


In [98]:
simlex_data.head()

,word1,word2,POS,path_similarity,cos_similarity
0,old,new,A,0.333333,0.441964
1,smart,intelligent,A,0.333333,0.704955
2,hard,difficult,A,1.000000,0.631380
3,happy,cheerful,A,0.333333,0.545871
4,hard,easy,A,0.333333,0.486345


In [99]:
None in simlex_data['cos_similarity']

False

# 4. Conduct experiments with another WordNet-based similarity implemented in the NLTK.

In [101]:
def word_similarity_wordnet_other(word1, word2):
    synsets1 = wn.synsets(word1)
    synsets2 = wn.synsets(word2)

    if not synsets1 or not synsets2:
        return None

    max_similarity = 0.0

    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.wup_similarity(synset2)  # Use a different similarity metric
            if similarity is not None and similarity > max_similarity:
                max_similarity = similarity

    return max_similarity

simlex_data['wup_similarity'] = [word_similarity_wordnet_other(row['word1'], row['word2']) for _, row in simlex_data.iterrows()]

/var/folders/9p/vfj51s1x3f7gl6dvkpvb09mm0000gn/T/ipykernel_42659/3429839475.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simlex_data['wup_similarity'] = [word_similarity_wordnet_other(row['word1'], row['word2']) for _, row in simlex_data.iterrows()]


In [102]:
simlex_data.head()

,word1,word2,POS,path_similarity,cos_similarity,wup_similarity
0,old,new,A,0.333333,0.441964,0.5
1,smart,intelligent,A,0.333333,0.704955,0.5
2,hard,difficult,A,1.000000,0.631380,1.0
3,happy,cheerful,A,0.333333,0.545871,0.5
4,hard,easy,A,0.333333,0.486345,0.5


In [103]:
min(simlex_data['wup_similarity'])

0.1

In [104]:
simlex_data[simlex_data['wup_similarity'] == 0.1]

,word1,word2,POS,path_similarity,cos_similarity,wup_similarity
225,dictionary,definition,N,0.052632,0.533919,0.1


In [105]:
max(simlex_data['wup_similarity'])

1.0

In [106]:
simlex_data[simlex_data['wup_similarity'] == 1]

,word1,word2,POS,path_similarity,cos_similarity,wup_similarity
2,hard,difficult,A,1.0,0.631380,1.0
6,happy,glad,A,1.0,0.673587,1.0
19,hard,tough,A,1.0,0.662226,1.0
20,new,fresh,A,1.0,0.446335,1.0
24,wonderful,terrific,A,1.0,0.773164,1.0
...,...,...,...,...,...,...
839,acquire,get,V,1.0,0.381536,1.0
843,take,carry,V,1.0,0.449714,1.0
861,steal,buy,V,1.0,0.462597,1.0
885,declare,announce,V,1.0,0.507106,1.0


# 5. Conduct experiments with another fastText English model from the list https://fasttext.cc/docs/en/english-vectors.html .

In [108]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec', binary=False)

def word_similarity_wiki(word1, word2, model):
    try:
        vector1 = model[word1]
        vector2 = model[word2]
        cosine_similarity = 1 - scipy.spatial.distance.cosine(vector1, vector2)
        return cosine_similarity
    except KeyError:
        return None

# Calculate similarities and add them to the DataFrame
simlex_data['wiki'] = [word_similarity_wiki(row['word1'], row['word2'], model) for _, row in simlex_data.iterrows()]

/var/folders/9p/vfj51s1x3f7gl6dvkpvb09mm0000gn/T/ipykernel_42659/1299633981.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simlex_data['wiki'] = [word_similarity_wiki(row['word1'], row['word2'], model) for _, row in simlex_data.iterrows()]


In [109]:
simlex_data.head()

,word1,word2,POS,path_similarity,cos_similarity,wup_similarity,wiki
0,old,new,A,0.333333,0.441964,0.5,0.653033
1,smart,intelligent,A,0.333333,0.704955,0.5,0.768561
2,hard,difficult,A,1.000000,0.631380,1.0,0.701083
3,happy,cheerful,A,0.333333,0.545871,0.5,0.663085
4,hard,easy,A,0.333333,0.486345,0.5,0.645981


# 6. Calculate Kendall’s tau (e.g. using scipy.stats.kendalltau) between the gold standard and obtained scores (use only word pairs processed by all models). Summarize findings in a table and analyze them.

In [23]:
from scipy.stats import kendalltau

kendall_results = pd.DataFrame(columns=['Model', 'Kendall Tau', 'p-value'])

for model_column in ['path_similarity', 'cos_similarity', 'wup_similarity', 'wiki']:
    common_pairs_mask = ~simlex_data[model_column].isnull()
    tau, p_value = kendalltau(simlex_data['SimLex999'][common_pairs_mask], simlex_data[model_column][common_pairs_mask])
    kendall_results = kendall_results.append({'Model': model_column, 'Kendall Tau': tau, 'p-value': p_value}, ignore_index=True)

kendall_results

/var/folders/9p/vfj51s1x3f7gl6dvkpvb09mm0000gn/T/ipykernel_42659/953755375.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kendall_results = kendall_results.append({'Model': model_column, 'Kendall Tau': tau, 'p-value': p_value}, ignore_index=True)
/var/folders/9p/vfj51s1x3f7gl6dvkpvb09mm0000gn/T/ipykernel_42659/953755375.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kendall_results = kendall_results.append({'Model': model_column, 'Kendall Tau': tau, 'p-value': p_value}, ignore_index=True)
/var/folders/9p/vfj51s1x3f7gl6dvkpvb09mm0000gn/T/ipykernel_42659/953755375.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kendall_results = kendall_results.append({'Model': model_column, 'Kendall Tau': tau, 'p-value': p_valu

,Model,Kendall Tau,p-value
0,path_similarity,0.352995,1.713205e-54
1,cos_similarity,0.330140,7.744003e-55
2,wup_similarity,0.319578,2.896237e-49
3,wiki,0.322270,2.441105e-52
